# Proyecto Renfe - Data Analytics y Business Intelligence

Recibimos dos datasets:

1. `renfe.csv`: Información de búsquedas de billetes que se hicieron en la página de Renfe.
2. `coordenadas_ciudades.csv`: Latitud y longitud de provincias españolas.

Queremos usar estos datasets para un modelo de Machine Learning que utilizaremos para predecir los precios de los billetes. Y, para ello, necesitamos limpiar, explorar y pre-procesar el dataset.

## Reglas de juego

1. El proyecto se debe entregar en grupos de dos o individualmente. 
2. Cada respuesta correcta suma un punto.
3. La calificación final consistirá en la suma de todos los puntos obtenidos sobre el total de puntos posibles.


## Diccionario de datos

Esta es la información provista:

### `renfe.csv`
- `FECHA_CONSULTA`: Fecha en la que se consultó la página.
- `FECHA_INICIO`: Fecha de inicio del trayecto.
- `FECHA_FIN`: Fecha de finalización del trayecto.
- `CIUDAD_ORIGEN`: Ciudad de origen del trayecto.
- `CIUDAD_DESTINO`: Ciudad destino del trayecto.
- `TIPO_TREN`: Tipo de tren.
- `TIPO_TARIFA`: Tipo de tarifa del billete.
- `CLASE`: Clase del asiento seleccionado.
- `PRECIO`: Precio del tren seleccionado.

### `coordenadas_ciudades.csv`
- `ciudad`: Nombre de la ciudad.
- `latitud`: Coordenada de latitud de la ciudad.
- `longitud`: Coordenada de longitud de la ciudad.

## Importar librerías

In [ ]:
import numpy as np
import pandas as pd 
import plotly.express as px
import plotly.io as pio
from datetime import datetime
import folium
pd.set_option("display.max_columns", 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# Leer el dataset `renfe.csv`

In [ ]:
df = pd.read_csv("D:/Anaconda/Nuclio/Proyecto_renfe/data/renfe.csv", sep=";", encoding= "UTF-8")

In [ ]:
df

## Visualizar las primeras y las últimas filas del dataset

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

## Cambiar los nombres de todas las columnas a minúsculas

In [ ]:
#Reviso el nombre de las columnas
df.columns

In [ ]:
#Reviso que mi codigo funciona como quiero que funcione
df.columns.str.lower()

In [ ]:
#Hago el cambio e imprimo
df.columns = df.columns.str.lower()
df.columns

In [ ]:
#Reviso que el dataframe se ve como espero que se vea
df.head()

## Mostrar los tipos de datos de cada columna

In [ ]:
df.info()

In [ ]:
#Reviso en general cuantos nan va a haber
df.isna().sum()

## Cambiar los tipos de datos, por los tipos adecuados

In [ ]:
#Reviso que el codigo funcione
pd.to_datetime(df['fecha_fin'])

In [ ]:
#Aplico en cada columna
df['fecha_consulta'] = pd.to_datetime(df['fecha_consulta'])
df['fecha_inicio'] = pd.to_datetime(df['fecha_inicio'])
df['fecha_fin'] = pd.to_datetime(df['fecha_fin'])
df.info()

## Tratamiento de filas duplicadas

In [ ]:
#Reviso cuantas filas duplicada hay
df.duplicated().sum()

In [ ]:
#Las ordeno para ver si hay algo raro, no encuentro nada mas que duplicidad, 
# que a menos que quiera contar el numero de solicitudes no representa problema eliminarlas
df.loc[df.duplicated(keep=False)].sort_values(by = 'fecha_consulta').reset_index()

### Quitar las filas duplicadas

In [ ]:
#Reviso como quedaria despues del borrado
df.shape, df.drop_duplicates().shape

In [ ]:
#Elimino duplicados
df = df.drop_duplicates()

In [ ]:
#Ordeno para tener mejor visualización de los datos
df.sort_values(by = 'fecha_consulta').reset_index(drop=True)

In [ ]:
#Aplico el cambio
df = df.sort_values(by = 'fecha_consulta').reset_index(drop=True) 

In [ ]:
#Doy un vistazo de como quedo ordenada y sin duplicados
df

## Valores nulos y análisis de `precio`

In [ ]:
#Busco nan y los sumo para que se muestren todas las columnas que tengan nulos.
df.isna().sum()

In [ ]:
#Los muestro todos juntos
df.tipo_tarifa.isnull().value_counts(normalize=True)*100,\
df.clase.isnull().value_counts(normalize=True)*100,\
df.precio.isnull().value_counts(normalize=True)*100

### Percentiles importantes (25%, 50%, 75%), minimo y máximo de `precio`?

In [ ]:
df.precio.describe()

In [ ]:
#Encontre 0s al describir en el paso anterior 
# y por los análisis que hice en la eliminación de duplicados deduje que solo los 0s eran posibles errores.
#Reviso que implica eliminarlos
df.shape, df.loc[df['precio'] != 0].shape

In [ ]:
#Aplico la eliminación
df = df.loc[df['precio'] != 0]

### Reemplazar los valores nulos en `precio` por la media de esa columna

In [ ]:
#Aqui parece que estoy haciendo una tonteria pero estoy revisando que si se reemplacen con este codigo
df['precio'].fillna(df.precio.mean()).isna().value_counts()

In [ ]:
#Aplico el cambio
df['precio'].fillna(df.precio.mean(),inplace=True)

In [ ]:
#Reviso todos las columnas para ver que ha cambiado
df.isna().sum()

### Eliminación de las filas donde `clase` o `tipo_tarifa` sean nulos

In [ ]:
#Reviso que coincidan los nulos de clase con los nulos en clase o tipo tarifa para hacer un solo eliminado
df.loc[df.clase.isna()].shape , df.loc[(df.clase.isna())|(df.tipo_tarifa.isna())].shape

In [ ]:
#Reviso que cambios van a suceder
df.shape, df.dropna(subset=['clase', 'tipo_tarifa']).shape

In [ ]:
#Aplico los cambios
df = df.dropna(subset=['clase', 'tipo_tarifa'])

## Tiempo de viaje

### Calculo el tiempo de viaje en minutos (fecha_fin - fecha_inicio)

In [ ]:
#Reviso las columnas y en que formato esta la información
df.head(3)

In [ ]:
#Creo una nueva columna y aplico el calculo
df['tiempo_de_viaje'] = (df['fecha_fin'] - df['fecha_inicio']).dt.total_seconds() / 60

In [ ]:
#Reviso que se creó
df.head(3)

In [ ]:
#Doy un vistazo para ver si hay algo extraño, me preocupa el viaje de 12 horas pero no creo que sea imposible
df.describe()

### Histograma de la variable que acabas de crear (`tiempo_de_viaje`)

In [ ]:
#Muestro la grafica simple y al ver otros viajes tambien de larga duración, no me preocupa mas el de 12 horas
df.plot(x='tiempo_de_viaje', kind='hist',backend='plotly',)

## Día, el nombre del día, el mes y la hora de `fecha_inicio`

In [ ]:
#Reviso rapido que todo se vea como lo espero
df['fecha_inicio'].dt.day ,df['fecha_inicio'].dt.day_name() ,df['fecha_inicio'].dt.month ,df['fecha_inicio'].dt.hour

In [ ]:
#Creo las nuevas columnas
df['dia'] = df['fecha_inicio'].dt.day
df['nombre_dia'] = df['fecha_inicio'].dt.day_name()
df['mes'] = df['fecha_inicio'].dt.month
df['hora'] = df['fecha_inicio'].dt.hour
df.head(3)

## Eliminar las columnas `fecha_consulta`, `fecha_inicio` y `fecha_fin` del dataset

In [ ]:
#Hago una copia, en caso de que vuelva a requerir estas columnas que voy a eliminar
df_respaldo = df.copy

In [ ]:
#Muestro las columnas para copiar y pegar en el codigo 
df.columns

In [ ]:
#Reviso como va a quedar
df.drop(['fecha_consulta', 'fecha_inicio', 'fecha_fin'],axis=1)

In [ ]:
#Aplico y doy un vistazo
df = df.drop(['fecha_consulta', 'fecha_inicio', 'fecha_fin'],axis=1)
df.head(3)

## Lectura del dataset `coordenadas_ciudades.csv` y unión con el dataset procesado hasta ahora (utiliza `ciudad_destino` para el `join`)

In [ ]:
df_coor = pd.read_csv("D:/Anaconda/Nuclio/Proyecto_renfe/data/coordenadas_ciudades.csv")
df_coor

In [ ]:
#Hago un join con valores por defecto
df2= df.join(df_coor.set_index('ciudad'), on='ciudad_destino')
df2.head(10)

## Gráfica en un mapa el precio medio por ciudad de destino

In [ ]:
#Agrupo y saco el precio medio
df2.groupby('ciudad_destino')['precio'].mean().reset_index()

In [ ]:
#Creo un pequeño DataFrame
precio_medio = df2.groupby('ciudad_destino')['precio'].mean().reset_index()
precio_medio

In [ ]:
#Creo otro pequeño DataFrame con .agg para agregar las dos columnas al mismo tiempo
df2.groupby('ciudad_destino').agg({'latitud': 'mean', 'longitud': 'mean'}).reset_index()

In [ ]:
#Creo otro pequeño DataFrame con .agg para agregar las dos columnas al mismo tiempo
coor = df2.groupby('ciudad_destino').agg({'latitud': 'mean', 'longitud': 'mean'}).reset_index()
coor

In [ ]:
#Hago un merge para unir todo
precio_medio.merge(coor, on='ciudad_destino')

In [ ]:
#Hago un merge para unir todo
precio_medio = precio_medio.merge(coor, on='ciudad_destino')
precio_medio

In [ ]:
#Hago el mapa con el centro de Madrid como centro y el zoom del tamaño que se vea bien
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

#Se dibujan los puntos de cada ciudad y sus caracteristicas
for index, row in precio_medio.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        popup=f"{row['ciudad_destino']}: {row['precio']:.2f} EUR",
        color="blue",
        weight=0,
        radius= row['precio']/2,
        fill=True,
        fill_opacity=0.6,
        opacity=1,
    ).add_to(mapa)

#Muestro el mapa
mapa.save('mapa_precio_medio.html')
mapa

## Tabla de correlación

In [ ]:
#Creo la matriz de correlación
matriz_de_correlacion = df2.corr(numeric_only=True)

In [ ]:
#Muestro la matriz para analizarla
matriz_de_correlacion

__________________________________________________________________________________________________________________________

#### Tiempo de viaje.
#### La correlación muestra una tendecia moderada negativa con respecto del tiempo de viaje lo que nos daria a entender que a medida que el tiempo de viaje aumenta el precio disminuye.

#### Mes.
#### La correlación muestra una tendecia moderada negativa con respecto del mes lo que nos daria a entender que a medida que el mes aumenta el precio disminuye.

#### Latitud y Longitud.
#### La correlación muestra una tendecia moderada positiva con respecto de longitud y latitud lo que nos daria a entender que a medida que el destino está mas al noreste, el precio es mas alto.

#### Dia y hora.
#### Los desestimo por tener una correlacion muy baja positiva.

#### Estas correlaciones con respecto del precio no son necesariamente absolutas y debido a que los valores son moderados no significan una realidad importante, solo marcan una ligera tendencia.


___________________________________________________________________________________________________________________________

## Relación entre variables del dataset y `precio`

### Scatter plot de precio vs. tiempo de viaje

In [ ]:
#Muestro columnas para copiar y pegar
df2.columns

In [ ]:
#Muestro las graficas por ciudad destino desde ciudad orgien
px.scatter(df2,x='precio',y='tiempo_de_viaje',labels={'ciudad_origen':'Origen',\
           'ciudad_destino':'Destino','precio':'Precio', 'tiempo_de_viaje':'Duración'},\
            color='ciudad_destino', facet_col='ciudad_origen',facet_col_wrap=3,\
            hover_data=['ciudad_origen','ciudad_destino','tipo_tarifa','tipo_tren'])

### Boxplot de precio vs. dia de la semana

In [ ]:
#Muestro las columnas para copiar y pegar
df2.columns

In [ ]:
#ordeno los días de la semana en el DataFrame
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df2['nombre_dia'] = pd.Categorical(df2['nombre_dia'], categories=dias_ordenados, ordered=True)
df2 = df2.sort_values('nombre_dia')

#Muestro la grafica
px.box(df2, x='nombre_dia', y='precio', \
    category_orders={'nombre_dia': dias_ordenados},labels={'nombre_dia':'Días de la semana', 'precio':'Precio'}, \
    hover_data=['ciudad_destino', 'tipo_tren', 'tipo_tarifa', 'clase'])

### Gráfica el precio medio por día de la semana

In [ ]:
#Agrúpo y hago la columna de precio medio
df2.groupby('nombre_dia').agg({'precio': 'mean'}).reset_index()

In [ ]:
#Aplico el cogido a una variable
p_medio_dia_semana = df2.groupby('nombre_dia').agg({'precio': 'mean'}).reset_index()
#Hago la grafica
fig = px.bar(p_medio_dia_semana, x='nombre_dia', y='precio', title='Precios por día de la semana', text='precio', 
             labels={'nombre_dia': 'Día de la semana', 'precio': 'Precio medio'}, 
             category_orders={'nombre_dia': dias_ordenados})
#Acomodo las etiquetas de precio para que se muestren en las barras y las etiquetas de los axis
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', yaxis_title='Precio', xaxis_title='Día de la semana')

fig.show()

## Nuevo dataframe *one-hot-encoding* a las variables categoricas

In [ ]:
#Reviso el DataFrame para ver que columnas serian mejor para hacerlas categoricas
df2.head(10)

In [ ]:
#Importo onhotencoder de sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder
#Selecciono las columnas y las guardo en una variable
categorical_columns = ['ciudad_origen', 'ciudad_destino', 'tipo_tren', 'tipo_tarifa', 'clase','nombre_dia']
#Hago un nuevo DataFrame con las columnas categorica y las presento en 1 y 0 por que me gusta mas que "True" y "False" 
df_encoded = pd.get_dummies(df2, columns=categorical_columns, dtype= int)

df_encoded